# **Lab: Model Interpretation**


## Exercise 2: Xgboost with Hyperopt

We will train a Xgboost model on the same dataset as previously and explore Hyperopt for automatic hyperparameter tuning.

The steps are:
1.   Create new Git branch
2.   Load the dataset
3.   Train Xgboost model with default hyperparameter
4.   Hyperparameter tuning with Hyperopt
5.   Push changes


### 1.   Create new Git branch


**[1.1]** Create a new git branch called `adv_mla_3_xgboost`


In [ ]:
# Placeholder for student's code (command line)

In [ ]:
# Solution:
git checkout -b adv_mla_3_xgboost

**[1.2]** Launch Jupyter Lab from your virtual environment

In [ ]:
# Placeholder for student's code (command line)

In [ ]:
# Solution:
git checkout -b adv_mla_3_adaboost

**[1.3]** Navigate the folder `notebooks` and create a new jupyter notebook called `2_xgboost_hyperopt.ipynb`

### 2. Load the dataset

**[2.1]** Launch magic commands to automatically reload modules

In [ ]:
# Placeholder for student's code (Python code)

In [1]:
# Solution
%load_ext autoreload
%autoreload 2

**[2.2]** Import the pandas, numpy packages and dump from joblib

In [ ]:
# Placeholder for student's code (Python code)

In [2]:
# Solution
import pandas as pd
import numpy as np
from joblib import dump

**[2.3]** Import the `load_sets()` function from your custom package

In [ ]:
# Placeholder for student's code (Python code)

In [22]:
# Solution
from my_krml_149874.data.sets import load_sets

**[2.4]** Load the saved sets from `data/processed`

In [ ]:
# Placeholder for student's code (Python code)

In [23]:
#Solution:
X_train, y_train, X_val, y_val, X_test, y_test = load_sets(path='../data/processed/')

# 3. Train Xgboost model

**[3.1]** Import the xgboost package as xgb


In [ ]:
# Placeholder for student's code (Python code)

In [5]:
# Solution:
import xgboost as xgb

**[3.2]** Instantiate the XGBClassifier class into a variable called rf with random_state=8

In [ ]:
# Placeholder for student's code (Python code)

In [6]:
# Solution
xgboost1 = xgb.XGBClassifier()

**[3.3]** Import the function `fit_assess_classifier` from your custom package

In [ ]:
# Placeholder for student's code (Python code)

In [24]:
# Solution
from my_krml_149874.models.performance import fit_assess_classifier

**[3.4]** Fit the model and display its performance on the training and validation sets

In [ ]:
# Placeholder for student's code (Python code)

In [25]:
# Solution
xgboost1 = fit_assess_classifier(xgboost1, X_train, y_train, X_val, y_val)

Accuracy Training: 0.9239609230015057
F1 Training: 0.9236854561126538
Accuracy Validation: 0.9070568709426668
F1 Validation: 0.9065289905847367


**[3.5]** Import `dump` from `joblib` and save the fitted model into the folder `models` as a file called `xgboost_default`

In [ ]:
# Placeholder for student's code (Python code)

In [26]:
# Solution:
from joblib import dump

dump(xgboost1,  '../models/xgboost_default.joblib')

['../models/xgboost_default.joblib']

#4. Hyperparameter tuning with Hyperopt

**[4.1]** Import Trials, STATUS_OK, tpe, hp, fmin from hyperopt package

In [ ]:
# Placeholder for student's code (Python code)

In [27]:
# Solution:
from hyperopt import Trials, STATUS_OK, tpe, hp, fmin

**[4.2]** Define the search space for xgboost hyperparameters

In [34]:
space = {
    'max_depth' : hp.choice('max_depth', range(3, 15, 1)),
    'min_child_weight' : hp.choice('min_child_weight', range(3, 15, 1)),
    'learning_rate' : hp.quniform('learning_rate', 0.01, 0.4, 0.05),    
    'subsample' : hp.quniform('subsample', 0.7, 1, 0.05),
    'colsample_bytree' : hp.quniform('colsample_bytree', 0.2, 0.5, 0.05)
}

**[4.3]** Define a function called `objective` with the following logics:
- input parameters: hyperparameter seacrh space (`space`)
- logics: train a xgboost model with the search space and calculate the average accuracy score for cross validation with 5 folds
- output parameters: dictionary with the loss score and STATUS_OK

In [ ]:
# Placeholder for student's code (Python code)

In [29]:
# Solution:
def objective(space):
    from sklearn.model_selection import cross_val_score

    xgboost = xgb.XGBClassifier(
        max_depth = int(space['max_depth']),
        learning_rate = space['learning_rate'],
        min_child_weight = space['min_child_weight'],
        subsample = space['subsample'],
        colsample_bytree = space['colsample_bytree']
    )

    acc = cross_val_score(xgboost, X_train, y_train, cv=5, scoring="accuracy").mean()

    return{'loss': 1-acc, 'status': STATUS_OK }

**[4.4]** Launch Hyperopt search and save the result in a variable called `best`

In [35]:
best = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    max_evals=3
)

100%|█████████████████████████████| 3/3 [04:20<00:00, 86.96s/trial, best loss: 0.10055674218284949]


**[4.5]** Print the best set of hyperparameters

In [ ]:
# Placeholder for student's code (Python code)

In [36]:
# Solution:
print("Best: ", best)

Best:  {'colsample_bytree': 0.4, 'learning_rate': 0.35000000000000003, 'max_depth': 8, 'min_child_weight': 11, 'subsample': 0.75}


**[4.6]** Instantiate a XGBClassifier with best set of hyperparameters

In [ ]:
# Placeholder for student's code (Python code)

In [37]:
# Solution:
xgboost2 = xgb.XGBClassifier(
    max_depth = best['max_depth'],
    learning_rate = best['learning_rate'],
    min_child_weight = best['min_child_weight'],
    subsample = best['subsample'],
    colsample_bytree = best['colsample_bytree']
)

**[4.7]** Fit the model and display its performance on the training and validation sets

In [ ]:
# Placeholder for student's code (Python code)

In [38]:
# Solution:
xgboost2 = fit_assess_classifier(xgboost2, X_train, y_train, X_val, y_val)

Accuracy Training: 0.9301656220455898
F1 Training: 0.929947013068367
Accuracy Validation: 0.9036324292527154
F1 Validation: 0.9031038783192644


**[4.8]** Save the fitted model into the folder models as a file called `xgboost_best`

In [ ]:
# Placeholder for student's code (Python code)

In [39]:
# Solution:
dump(xgboost2,  '../models/xgboost_best.joblib')

['../models/xgboost_best.joblib']

# 5.   Push changes

**[5.1]** Add you changes to git staging area

In [ ]:
# Placeholder for student's code (command line)

In [ ]:
# Solution:
git add .

**[5.2]** Create the snapshot of your repository and add a description

In [ ]:
# Placeholder for student's code (command line)

In [ ]:
# Solution:
git commit -m "xgboost hyperopt"

**[5.3]** Push your snapshot to Github


In [ ]:
# Placeholder for student's code (command line)

In [ ]:
# Solution:
git push

**[5.4]** Go to Github and merge the branch after reviewing the code and fixing any conflict




**[5.5]** Check out to the master branch

In [ ]:
# Placeholder for student's code (command line)

In [ ]:
# Solution:
git checkout master

**[5.6]** Pull the latest updates

In [ ]:
# Placeholder for student's code (command line)

In [ ]:
# Solution
git pull